In [8]:
import pandas as pd
import json
import torch
from torch.utils.data import Dataset
from torch.serialization import add_safe_globals

from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback
)

import numpy as np
from seqeval.metrics import classification_report, f1_score

In [ ]:
# Load Train/Val/Test

train_df = pd.read_csv("dataset/snips_ner_train.csv")
val_df   = pd.read_csv("dataset/snips_ner_val.csv")
test_df  = pd.read_csv("dataset/snips_ner_test.csv")

for df in (train_df, val_df, test_df):
    df["token"] = df["token"].astype(str)

print("Train:", train_df.shape)
print("Val:  ", val_df.shape)
print("Test: ", test_df.shape)

Train: (110572, 6)
Val:   (12191, 6)
Test:  (6318, 6)


In [ ]:
# Build Label Vocab

# Extract raw slot names without BIO prefixes
raw_slot_types = sorted({
    label.split("-", 1)[1] 
    for label in train_df["slot"].unique() 
    if label != "O"
})

# Build BIO tag set
full_labels = ["O"]
for slot in raw_slot_types:
    full_labels.append(f"B-{slot}")
    full_labels.append(f"I-{slot}")

print("Total BIO labels:", len(full_labels))

ner_label2id = {label: i for i, label in enumerate(full_labels)}
ner_id2label = {i: label for label, i in ner_label2id.items()}

# Save
with open("ner_label2id.json", "w") as f:
    json.dump(ner_label2id, f, indent=2)

with open("ner_id2label.json", "w") as f:
    json.dump(ner_id2label, f, indent=2)

print("Saved ner_label2id.json and ner_id2label.json")

num_labels = len(ner_label2id)

Total BIO labels: 79
Saved ner_label2id.json and ner_id2label.json


In [ ]:
# Reconstruct Sentences

def group_sentences(df):
    sentences = []
    for sid, group in df.groupby("sentence_id"):
        group = group.sort_values("token_id")
        tokens = group["token"].tolist()
        slots  = group["slot"].tolist()
        intent = group["intent"].iloc[0]
        sentences.append({
            "sentence_id": sid,
            "tokens": tokens,
            "slots": slots,
            "intent": intent,
        })
    return sentences

train_sents = group_sentences(train_df)
val_sents   = group_sentences(val_df)
test_sents  = group_sentences(test_df)

print(f"Train sentences: {len(train_sents)}")
print(f"Val sentences:   {len(val_sents)}")
print(f"Test sentences:  {len(test_sents)}")

Train sentences: 12405
Val sentences:   1379
Test sentences:  700


In [ ]:
# Tokenizer and Subword Alignment

MODEL_NAME = "distilbert-base-uncased" #<- Use This for Distilbert
# MODEL_NAME = "google-bert/bert-base-uncased"
MAX_LEN = 64

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def encode_ner(sentences, tokenizer, label2id, max_len=64):
    all_tokens = [s["tokens"] for s in sentences]
    all_slots  = [s["slots"]  for s in sentences]

    enc = tokenizer(
        all_tokens,
        is_split_into_words=True,
        padding="max_length",
        truncation=True,
        max_length=max_len,
        return_attention_mask=True,
    )

    encoded_labels = []

    for i, slots in enumerate(all_slots):
        word_ids = enc.word_ids(batch_index=i)
        labels_i = []
        previous_wid = None

        for wid in word_ids:
            if wid is None:
                labels_i.append(-100)
            else:
                label = slots[wid]

                # First subword → keep B- or I-
                if wid != previous_wid:
                    labels_i.append(label2id[label])
                else:
                    # Continuation subwords → ALWAYS I-tag
                    if label.startswith("B-"):
                        label = "I-" + label[2:]
                    labels_i.append(label2id[label])

                previous_wid = wid

        encoded_labels.append(labels_i)

    labels_tensor = torch.tensor(encoded_labels, dtype=torch.long)
    enc_dict = {k: v for k, v in enc.items()}

    return enc_dict, labels_tensor


train_encodings, train_labels = encode_ner(train_sents, tokenizer, ner_label2id)
val_encodings,   val_labels   = encode_ner(val_sents,   tokenizer, ner_label2id)
test_encodings,  test_labels  = encode_ner(test_sents,  tokenizer, ner_label2id)

In [ ]:
# Dataset Classes for SNIPS NER

class SnipsNerDataset(Dataset):
    def __init__(self, encodings, labels):
        self.enc = encodings
        self.labels = labels

    def __len__(self):
        return self.labels.shape[0]

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.enc.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

add_safe_globals([SnipsNerDataset])

train_dataset = SnipsNerDataset(train_encodings, train_labels)
val_dataset   = SnipsNerDataset(val_encodings, val_labels)
test_dataset  = SnipsNerDataset(test_encodings, test_labels)

In [ ]:
# Model

model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    id2label=ner_id2label,
    label2id=ner_label2id,
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Training Arugments

training_args = TrainingArguments(
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    warmup_ratio=0.1,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=20,
    load_best_model_at_end=True,
    seed=42, # reproducibility
)

In [16]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)

    # Token-level accuracy (optional)
    true_flat = []
    pred_flat = []
    for p_seq, l_seq in zip(preds, labels):
        for p, l in zip(p_seq, l_seq):
            if l == -100:
                continue
            true_flat.append(l)
            pred_flat.append(p)

    acc = (np.array(true_flat) == np.array(pred_flat)).mean()

    return {"token_accuracy": acc}

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [18]:
trainer.train()

c:\Users\Pradhyumnaa G\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\Pradhyumnaa G\AppData\Local\Temp\ipykernel_16948\1056801481.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx])


Epoch,Training Loss,Validation Loss,Token Accuracy
1,0.468000,0.392973,0.914341
2,0.133300,0.126014,0.970074
3,0.091500,0.096522,0.973254
4,0.044500,0.098714,0.976001
5,0.013500,0.111213,0.974772
6,0.016900,0.120831,0.975351


c:\Users\Pradhyumnaa G\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\Pradhyumnaa G\AppData\Local\Temp\ipykernel_16948\1056801481.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx])
c:\Users\Pradhyumnaa G\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\Pradhyumnaa G\AppData\Local\Temp\ipykernel_16948\1056801481.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourc

TrainOutput(global_step=4656, training_loss=0.37501098021314416, metrics={'train_runtime': 4130.0508, 'train_samples_per_second': 45.054, 'train_steps_per_second': 2.818, 'total_flos': 1217256153281280.0, 'train_loss': 0.37501098021314416, 'epoch': 6.0})

In [ ]:
# SEQUEVAL Evaluation

def decode_predictions(encodings, labels, preds, id2label):
    y_true = []
    y_pred = []

    for i in range(labels.shape[0]):
        sent_true = []
        sent_pred = []

        for p, l in zip(preds[i], labels[i]):
            if l == -100:
                continue
            sent_true.append(id2label[int(l)])
            sent_pred.append(id2label[int(p)])

        y_true.append(sent_true)
        y_pred.append(sent_pred)

    return y_true, y_pred

In [20]:
preds = trainer.predict(test_dataset)
logits = preds.predictions
pred_ids = np.argmax(logits, axis=-1)

y_true, y_pred = decode_predictions(
    test_encodings, test_labels, pred_ids, ner_id2label
)

print("\n===== SEQEVAL (ENTITY-LEVEL) REPORT =====")
print(classification_report(y_true, y_pred))
print("Entity F1:", f1_score(y_true, y_pred))

c:\Users\Pradhyumnaa G\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\Pradhyumnaa G\AppData\Local\Temp\ipykernel_16948\1056801481.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item["labels"] = torch.tensor(self.labels[idx])



===== SEQEVAL (ENTITY-LEVEL) REPORT =====
                            precision    recall  f1-score   support

                     album       0.00      0.00      0.00        13
                    artist       0.93      0.95      0.94       109
               best_rating       1.00      1.00      1.00        51
                      city       0.93      0.92      0.92        71
     condition_description       0.96      1.00      0.98        22
     condition_temperature       1.00      1.00      1.00        21
                   country       0.98      0.95      0.97        44
                   cuisine       1.00      0.91      0.95        11
          current_location       1.00      0.94      0.97        17
               entity_name       0.65      0.72      0.68        18
                  facility       1.00      1.00      1.00         7
                     genre       0.29      0.67      0.40         3
            geographic_poi       1.00      1.00      1.00        16
    

In [21]:
trainer.save_model("snips_ner_model_full")
tokenizer.save_pretrained("snips_ner_model_full")

('snips_ner_model_full\\tokenizer_config.json',
 'snips_ner_model_full\\special_tokens_map.json',
 'snips_ner_model_full\\vocab.txt',
 'snips_ner_model_full\\added_tokens.json',
 'snips_ner_model_full\\tokenizer.json')